# Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Import libs

In [1]:
import pandas as pd

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

import numpy as np

## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

In [2]:
!mkdir -p data
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o data/yellow_tripdata_2023-01.parquet
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -o data/yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  23.1M      0  0:00:01  0:00:01 --:--:-- 23.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0  24.6M      0  0:00:01  0:00:01 --:--:-- 24.6M


In [3]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

Read the data for January. How many columns are there?

In [4]:
len(df.columns)

19

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

In [5]:
df.dtypes
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

What's the standard deviation of the trips duration in January?

In [6]:
arr = np.array(df.duration)
float(round(arr.std(), 2))

42.59

# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
float(round(((df.duration >= 1) & (df.duration <= 60)).mean(), 2) * 100)

98.0

# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

In [8]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

Fit a dictionary vectorizer

In [9]:
dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()

Get a feature matrix from it

In [10]:
x_train = dv.fit_transform(dicts)

What's the dimensionality of this matrix (number of columns)?

In [11]:
x_train.shape[1] - 1

515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable

In [12]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)

Calculate the RMSE of the model on the training data

In [13]:
rmse = root_mean_squared_error(y_train, y_pred)

What's the RMSE on train?

In [14]:
float(round(rmse, 2))

7.66

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

In [15]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [16]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_val[categorical] = df_val[categorical].astype(str)
dicts_val = df_val[categorical + numerical].to_dict(orient='records')

x_val = dv.transform(dicts_val)

target = 'duration'
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_val)
val_rmse = root_mean_squared_error(y_val, y_pred)

What's the RMSE on validation?

In [17]:
float(round(val_rmse, 2))

7.82